In [1]:
# import neccessary stuff 
#run this from the datascience-films folder
import pandas as pd 
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
# ignore some warnings, not pressing to fix these issues, more annoying to show these
from pandas.core.common import SettingWithCopyWarning
import warnings
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

RESOURCE_PATH = Path('../cleaned_data')

df = pd.read_csv(RESOURCE_PATH / 'Export_historical_vietnam_v2.csv',sep=',')
df['movie_release_year'] = df['movie_release_year'].fillna(0)

In [2]:
df['numVotes'] = df['numVotes'].fillna(0)
#lastly pop our target we want
target = df[['movie_release_year','averageRating','numVotes']]
target.averageRating =  target.averageRating.fillna(5)
target['numVotes'] = target['numVotes'].fillna(0)
target['numVotes'] = target['numVotes'].astype(np.int64)
target['averageRating'] = target['averageRating'].fillna(0)


In [3]:
# split genres per genre and create column per genre
unique_genres = pd.unique(df["genres"].str.split(",", expand=True).stack())
df = df.join(df.pop('genres').str.get_dummies(sep=',').astype(bool))
df = df.join(df.pop('historical_event').str.join('').str.get_dummies().astype(bool))

In [4]:
#get the events 

# get numeric features and set as target
numeric_feature_names = list(unique_genres)
numeric_feature_names.append('vietnam war')
numeric_feature_names.append('second world war')
numeric_features = df[numeric_feature_names]
numeric_features

,Comedy,Short,Music,Romance,Sci-Fi,Adventure,Documentary,Action,Biography,Drama,...,Horror|War,Action|Drama|Romance,(no genres listed),Thriller|War,Documentary|Drama,Drama|Thriller,Action|Thriller,Comedy|Drama|Romance,vietnam war,second world war
0,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,False,True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,True,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,False,True,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
288,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
289,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
290,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [5]:
from sklearn.model_selection import train_test_split

# splits the training and test data set in 65% : 35%
# assign random_state to any value.This ensures consistency.
X_train, X_test, Y_train, Y_test = train_test_split(numeric_features, target, test_size = 0.3, random_state=20)
X_valid, X_test, Y_valid, Y_test = train_test_split(X_test,Y_test,test_size=0.5)
print(X_train.shape)
print(Y_train.shape)

print(X_test.shape)
print(Y_test.shape)

print(X_valid.shape)
print(Y_valid.shape)

(204, 53)
(204, 3)
(44, 53)
(44, 3)
(44, 53)
(44, 3)


In [6]:
#import linear regression model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=5, include_bias=False)
poly_features = poly.fit_transform(X_train)

linear_model = LinearRegression()
linear_model.fit(X_train, Y_train)

y_train_predict = linear_model.predict(X_train)

rmse = (np.sqrt(mean_squared_error(Y_train, y_train_predict)))
r2 = r2_score(Y_train, y_train_predict)

print("The model performance for training set")
print("--------------------------------------")
print(f'RMSE is {rmse}')
print(f'R2 score is {r2}')
print("\n")

y_test_predict = linear_model.predict(X_test)
# root mean square error of the model
rmse = (np.sqrt(mean_squared_error(Y_test, y_test_predict)))

# r-squared score of the model
r2 = r2_score(Y_test, y_test_predict)

print("The model performance for testing set")
print("--------------------------------------")
print(f'RMSE is {rmse}')
print(f'R2 score is {r2}')
print("This model is slightly suitable for predicting the release after the event")

# Visualizing the data

In [10]:
plt.scatter(diabetes_X_test, diabetes_y_test, color="black")
plt.plot(diabetes_X_test, diabetes_y_pred, color="blue", linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()

/home/shivan/school/jaar_4/datascience-films/env/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


ValueError: X has 16 features, but LinearRegression is expecting 27 features as input.